In [ ]:
from flask import Flask, render_template, url_for, request, flash, redirect, jsonify, session
from flask_sqlalchemy import SQLAlchemy
import json
import os
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime


app = Flask(__name__)
app.secret_key = 'KEY'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)

# The following line ensures that the above environment variables are available in the
# application context as needed.
# https://stackoverflow.com/questions/31444036/runtimeerror-working-outside-of-application-context
app.app_context().push()

# Database Models
class BlogPost(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    title = db.Column(db.String(), nullable = False)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)
    
    def __repr__(self):
        return '<Task %r>' % self.id
    
    @property
    def serialize(self):
        """
        Return the blog post as a dictionary. 
        """
        return {
            'id': self.id,
            'title': self.title,
            'content': self.content,
            'date_uploaded': self.date_uploaded,
        }
    
class Admin(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(120),nullable=False, unique = True)
    password = db.Column(db.String(120),nullable=False)


# Utility Functions
def create_admin_user():
    """
    Create an admin user with the default credentials.
    """
    default_admin=Admin(username = 'admin',password = 'password')
    db.session.add(default_admin)
    db.session.commit()

def add_post(post_title,post_content):
    """
    Add a blog post to the database.
    """
    new_blog=BlogPost(title=post_title,content=post_content)
    db.session.add(new_blog)
    db.session.commit()

def delete_post(blog_id):
    """
    Delete a post from the database
    """
    post = BlogPost.query.get_or_404(blog_id)
    db.session.delete(post)
    db.session.commit()

def edit_post(blog_id, new_title, new_content):
    """
    Update the title and content of a blog post in the database.
    """
    post = BlogPost.query.get_or_404(blog_id)
    post.title = new_title
    post.content = new_content
    db.session.commit()

def retrieve_blogs():
    """
    Return an iterable list of the blogs in the database.
    """
    blogs = BlogPost.query.all()
    serialized_data = []
    for post in blogs:
        serialized_data.append(post.serialize)
    return serialized_data

def retr_single_blog(blog_id):
    """
    Return a single serialized blog post. 
    """
    blog = BlogPost.query.get_or_404(blog_id)
    return blog.serialize

def initialize_database():
    """
    If a database does not exist, create a new database and populate it with default data.
    """
    try:
       admin = Admin.query.filter_by(username='admin').first() 
       print("Starting server with existing database.")
    except:
        print("No database found. Creating default database now.")
        with app.app_context():
            db.create_all()

        create_admin_user()

        with open('sampleposts.json') as sample_posts:
            default_posts = json.loads(sample_posts.read())['blog_posts']


        for post in default_posts:
            add_post(post['title'],post['content'])


# Routing
@app.route('/')
def home_page():
    return render_template('index.html')

@app.route('/articles')
def articles():
    posts = retrieve_blogs()
    return render_template('articles.html', posts_list = posts)

# Admin interface, and adding a blog post to the database
@app.route('/admin', methods=('GET','POST'))
def admin():
    if 'authenticated' not in session or not session['authenticated']:
        return redirect('/login')
    elif request.method == 'POST':
        if request.form['form_id'] == 'add_post':
            post_title = request.form['title']
            post_content = request.form['content']
            add_post(post_title, post_content)
            flash(f'Blog post "{post_title}" added successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'delete_post':
            post_title = request.form['title']
            post_id = request.form['blog_id']
            delete_post(post_id)
            flash(f'Blog post "{post_title}" deleted successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'edit_post':
            post_data = retr_single_blog(request.form['blog_id'])
            return render_template('edit.html', post = post_data)
        
        elif request.form['form_id'] == 'post_editor':
            post_id = request.form['blog_id']
            new_title = request.form['title']
            new_content = request.form['content']
            edit_post(post_id, new_title, new_content)
            flash(f'Post ID# {post_id}: "{new_title}" has been edited successfully!')
            return redirect('/admin')
        
        elif request.form['form_id'] == 'logout':
            session['authenticated'] = False
            return redirect('/')
    else:
        posts = retrieve_blogs()
        return render_template('admin.html', posts_list = posts)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        admin = Admin.query.filter_by(username=username).first()

        if admin and admin.password == password:
            session['authenticated'] = True
            return redirect('/admin')
        else:
            flash("Error: Username or password incorrect")
            return render_template('login.html')
    else:
        return render_template('login.html')


    


# Error Code Handling
@app.errorhandler(404)
def not_found(e):
    return render_template('404.html'), 404

def main():
    initialize_database()    
    app.run()



if __name__ == "__main__":
    main()